# LOAD PACKAGES FOR PROJECT

In [ ]:
from bs4 import BeautifulSoup as Soup
import bs4
import pandas as pd
import import_ipynb
import numpy as np
import requests
from datetime import datetime
import url_file
from datetime import timedelta

# HOUSE PRICE DATA EXTRACTION :
## DATA WAS SCRAPED FROM :" - Website link details embeded in url_file for privacy

In [ ]:
def preview():
    for i in range(0,1):
      url=url_file.url
      r=requests.get(url).text
      soup=Soup(r,'html.parser')
      return soup.prettify()

preview()

In [ ]:
image_url=[]
Description=[]
Address=[]
Date=[]
Price=[]
Bedroom=[]
Bathroom=[]
Toilet=[]
for i in range(0,800):
    url=url_file.url+str(i)
    r=requests.get(url).text
    soup=Soup(r,'html.parser')
    wal=soup.find('div',{'class':'result-listings'})
    for info in wal.find_all('div',{'class':'similar-listings-info'}):
         description=info.find('h2')
         Description.append(description.text)
         address=info.find('p',{'class':'listings-location'})
         Address.append(address.text)
         date=info.find('div',{'class':'media'}).find('div',{'class':'media-body'}).find('h5',{'class':'mt-0'})
         Date.append(date.text)
    for info2 in wal.find_all('div',{'class':'similar-listings-price'}):
         price=info2.find('h4')
         Price.append(price.text)
         rooms=info2.find_all('ul',{'class':'property-benefit'})
         for room in rooms:
            col=room.find_all('li')
            bedroom=col[0].text
            bathroom=col[1].text
            toilet=col[2].text
            Bedroom.append(bedroom)
            Bathroom.append(bathroom)
            Toilet.append(toilet)

## DATA COLLECTION & DATA CLEANING
STORED IN A DICTIONARY CONVERTED TO DATAFRAME

In [ ]:
property_dict={"description":Description,
               "Address":Address,"Date":Date,"Price":Price,"Bedroom":Bedroom,
              "Bathroom":Bathroom,"Toilet":Toilet}
df=pd.DataFrame(property_dict)

In [ ]:
df[["date","cols"]]=df["Date"].str.split(',',expand=True)
df.drop(["Date","cols"],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df["date"]=df["date"].replace("Updated Today",datetime.now().strftime("%d %b %Y"))
df["date"]=df["date"].replace("Added Today",datetime.now().strftime("%d %b %Y"))
df["date"]=df["date"].replace("Updated Yesterday",(datetime.now()-timedelta(days = 1)).strftime("%d %b %Y"))
df["date"]=df["date"].replace({"Updated ":"","Added ":"","  ":""},regex=True)

In [ ]:
df["Toilet"].replace({"\n":""," ":""},regex=True,inplace=True)
df["Bedroom"].replace({"\n":""," ":""},regex=True,inplace=True)
df["Bathroom"].replace({"\n":""," ":""},regex=True,inplace=True)
df["Price"].replace({"/sqm":"","/day":"","/month":"","/year":"",",":""," ":""},regex=True,inplace=True)
df["Address"].replace({"\n":""},regex=True,inplace=True)
df=df.replace('',0,regex = True)

In [ ]:
df.head(4)

In [ ]:
df["Currency"]=df["Price"].str.slice(stop=1)
df["Price"].replace({"₦":"","$":"","$":""},regex=True,inplace=True)

In [ ]:
df["Bedroom"]=df["Bedroom"].astype('int')
df["Bathroom"]=df["Bathroom"].astype('int')
df["Toilet"]=df["Toilet"].astype('int')
df["date"]=pd.to_datetime(df["date"])

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df["date"]=pd.to_datetime(df["date"])
df["Price"]=df["Price"].astype('int')

In [ ]:
df["State"]=df["Address"].str.split().str[-1]
df["State"]=df["State"].replace({"4":"Abuja","Area":"Enugu"})
df["Location"]=df["Address"].str.split().str[-2]

In [ ]:
df.head(2)

In [ ]:
df.drop_duplicates

In [ ]:
df.describe(include="all")

In [ ]:
df.to_csv("originalhousedata.csv")

#  The Price column was converted to naira and currency column was deleted.

In [ ]:
df=pd.read_csv("originalhousedata.csv")

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df["date"]=pd.to_datetime(df["date"])
df.drop("Unnamed: 0", axis=1, inplace= True)

## EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
print(df[["Price","Bedroom","Bathroom","Toilet"]].corr())
dfplot= sns.heatmap(df[["Price","Bedroom","Bathroom","Toilet"]].corr(),annot=True)
plt.show()

In [ ]:
df["State"].value_counts().to_frame()

# Average price per state

In [ ]:
df1=df[["State","Price"]]
df1=df1.groupby(df1["State"]).mean().round(0)
df1.plot(kind='bar')
plt.ylabel("AVG. Price")
plt.title("Average price per state ")

# Maximum & minimum price per state (most expensive & least expensive houses prices per state)

In [ ]:
df2=df[["State","Price"]].where(df["Price"]>=400000)
df2i=df2.groupby(df2["State"]).min()
df2ii=df2.groupby(df2["State"]).max()

In [ ]:
print(df2i.sort_values(by="Price"))
df2i.sort_values(by="Price")[:10].plot(kind="bar",ylabel="Price(*10^6)",title="States with lowest minimum price")
df2i.sort_values(by="Price")[10:].plot(kind="bar",ylabel="Price(*10^8)",title="States with highest minimum price")

In [ ]:
print(df2ii.sort_values(by="Price"))
df2ii.sort_values(by="Price")[:10].plot(kind="bar",ylabel="Price(*10^8)",title="States with lowest maximum price")
df2ii.sort_values(by="Price")[10:].plot(kind="bar",ylabel="Price(*10^9)",title="States with highest maximum price")

# Frequent house description

In [ ]:
df["description"].value_counts()[:10]

In [ ]:
df["description"].value_counts()[:10].plot(kind="barh")

# Most expensive house description

In [ ]:
df2=df[["description","Price"]]
df2=df2.groupby(df2["description"]).max()[:10].sort_values(by="Price",ascending=False)
df2

# Post per date in year 2023

In [ ]:
df["date"].where((df["date"].dt.year)==2023).value_counts().plot(kind="line",ylabel="Number of posted sales",
                                                                 title="Sales posted per date",figsize=(15,10))

# Unique Locations

In [ ]:
df["Location"].value_counts()
location= list(df["Location"].unique())
unwanted=[1,2,3,4]
location=sorted(location, reverse = True)
del location[-4:]
location